In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
filename='video1.wav' 
import librosa 
x, sr = librosa.load(filename,sr=16000)

In [ ]:
import librosa


x, sr = librosa.load('video1.wav', sr=None)
duration = librosa.get_duration(y=x, sr=sr)
minutes = int(duration/60)

print(f"The duration of 'video1.wav' is {duration:.2f} seconds or {minutes} minutes.")




The duration of 'video1.wav' is 1698.71 seconds or 28 minutes.


In [ ]:
max_slice=5 
window_length = max_slice * sr

In [ ]:
import IPython.display as ipd 
a=x[21*window_length:22*window_length] 
ipd.Audio(a, rate=sr)

In [ ]:
energy = sum(abs(a**2))
print(energy)

4753.602325906511


In [ ]:
len(a)

240000

In [ ]:
import matplotlib.pyplot as plt 
fig = plt.figure(figsize=(14, 8)) 
ax1 = fig.add_subplot(211) 
ax1.set_xlabel('time') 
ax1.set_ylabel('Amplitude') 
ax1.plot(a)

In [ ]:
import numpy as np
energy = np.array([sum(abs(x[i:i+window_length]**2)) for i in range(0, len(x), window_length)])

In [ ]:
energy

In [ ]:
import matplotlib.pyplot as plt 
plt.hist(energy) 
plt.show()

In [ ]:
import pandas as pd
df=pd.DataFrame(columns=['energy','start','end'])
thresh=8000
row_index=0
for i in range(len(energy)):
  value=energy[i]
  if(value>=thresh):
    i=np.where(energy == value)[0]
    df.loc[row_index,'energy']=value
    df.loc[row_index,'start']=i[0] * 5
    df.loc[row_index,'end']=(i[0]+1) * 5
    row_index= row_index + 1

In [ ]:
df

In [ ]:
sum(energy >= thresh)

In [ ]:
temp=[]
i=0
j=0
n=len(df) - 2
m=len(df) - 1
while(i<=n):
  j=i+1
  while(j<=m):
    if(df['end'][i] == df['start'][j]):
      df.loc[i,'end'] = df.loc[j,'end']
      temp.append(j)
      j=j+1
    else:
      i=j
      break  
df.drop(temp,axis=0,inplace=True)

In [ ]:
df

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
import numpy as np

start = np.array(df['start'])
end = np.array(df['end'])

for i in range(len(df)):
    if i != 0:
        start_lim = start[i] - 5
    else:
        start_lim = start[i] 
    end_lim = end[i]   
    filename = '/content/drive/MyDrive/highlights/' + str(i+1) + '.mp4'
    ffmpeg_extract_subclip('/content/drive/MyDrive/video1.mp4', start_lim, end_lim, targetname=filename)
    print('Video clip generated:', filename)

In [ ]:
!pip install imageio_ffmpeg

In [ ]:
df

In [ ]:
import os
path = "/content/drive/MyDrive/highlights/"
files = os.listdir(path)
files.sort()
import subprocess
command = ['ffmpeg', '-y']
for file in files:
    if file.endswith('.mp4'):
        command += ['-i', os.path.join(path, file)]
command += ['-filter_complex', '[0:v]concat=n=%d:v=1[out]' % len(files),
            '-map', '[out]', '-c:v', 'libx264', '-crf', '23', '-preset', 'medium',
            os.path.join(path, 'merged.mp4')]
subprocess.run(command)
from google.colab import files
files.download(os.path.join(path, 'merged.mp4'))


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>